In [1]:
# Import library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import Image

# Open file
import os
import PIL
from random import seed
# Model CNN (Deep learning network)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense,\
GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
def plot_graph(model_values):
    ''' 
    Input : Model_values of keras.callbacks.History
    Return : Graph of Loss function and accuracy score between training dataset and vaildation dataset
    '''
    # Subplots
    fig, ax = plt.subplots(1, 2, figsize=(14,5))
    
    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(model_values.history['loss'], label='Training Loss');
    plt.plot(model_values.history['val_loss'], label='Testing Loss');
    plt.legend(fontsize=12, loc='upper right')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss');
    
    # Plot MSE
    plt.subplot(1, 2, 2)
    
    plt.plot(model_values.history['accuracy'], label='Training Accuracy')
    plt.plot(model_values.history['val_accuracy'], label='Validation Accuracy')
    
    plt.legend(fontsize=12, loc='lower right')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy');

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15345820738413896667
xla_global_id: -1
]


In [4]:
# set state seed
seed(42)

# Open path file of dataset 
dataset_path_new = "dataset_train_valid_test"

train_dir = os.path.join(dataset_path_new, "train")
valid_dir = os.path.join(dataset_path_new, "valid")
test_dir = os.path.join(dataset_path_new, "test")

In [5]:
# Set parameter of image 
# https://github.com/keras-team/keras/issues/8090#issuecomment-335155737
batch_size = 50 # Set the batch size for epoch cycle
img_height = 128 # Set the height of the picture
img_width = 128 # Set the width of the picture

# Rescale pixel to reduce image size before using in model
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)
data_gen_test = ImageDataGenerator(rescale=1/255.)

In [6]:
# Create training dataset 
train_dataset = data_gen_train.flow_from_directory(train_dir,
                                                   class_mode="categorical",
                                                   target_size=(img_height, img_width),
                                                   batch_size=batch_size)
 

# Create validation dataset 
valid_dataset = data_gen_valid.flow_from_directory(valid_dir,
                                                   class_mode="categorical",
                                                   target_size=(img_height, img_width),
                                                   batch_size=batch_size)

# Create testing dataset 
test_dataset = data_gen_test.flow_from_directory(test_dir,
                                                   class_mode="categorical",
                                                   target_size=(img_height, img_width),
                                                   batch_size=batch_size)

Found 6367 images belonging to 4 classes.
Found 686 images belonging to 4 classes.
Found 668 images belonging to 4 classes.


In [7]:
# https://keras.io/api/applications/mobilenet/#mobilenetv2-function
# import MobileNetV2 model form keras API
# set input size of image of trianing is 128x128 (smallest size of MobileNetV2)
# due to we want to use transfer learning process 
# we must add `include_top=False` because we wan to add our input data 
# we decide default weigh for mode
mobv2_model = tf.keras.applications.MobileNetV2(input_shape=(128,128,3),
                                                include_top=False, # Transfer learning
                                                weights="imagenet")

9406464/9406464 [==============================] - 4s 0us/step


In [8]:
mobv2_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][

In [9]:
mobv2_model.trainable=False

In [10]:
# Create output layer 
# We have 4 classes in our output we decide using activation="softmax" 
# for multi classification.
# Before output layer we decide use GlobalAveragePooling2D as 
# one type of flatten layer.
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(mobv2_model.output) # flatten
prediction_layer = tf.keras.layers.Dense(units=4, activation="softmax")(average_pooling_layer)

In [11]:
# Add Input layer and output layer 
model = tf.keras.models.Model(inputs=mobv2_model.input, 
                                    outputs=prediction_layer)

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][0]']           

In [14]:
# Compile the model         
model.compile(loss="categorical_crossentropy", 
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [15]:
# Save checkpoints during training
checkpoint_path = "../model/mobilenetV2/mobilenetv2_cp/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 mode="max",
                                                 verbose=1,
                                                 monitor="val_accuracy")

In [16]:
history = model.fit(train_dataset,
          epochs=25,
          validation_data=valid_dataset,
          callbacks=[cp_callback])

Epoch 1/25
128/128 [==============================] - ETA: 0s - loss: 0.4325 - accuracy: 0.8401
Epoch 1: saving model to ../model/mobilenetV2/mobilenetv2_cp\cp.ckpt
128/128 [==============================] - 371s 3s/step - loss: 0.4325 - accuracy: 0.8401 - val_loss: 0.2594 - val_accuracy: 0.9096
Epoch 2/25
128/128 [==============================] - ETA: 0s - loss: 0.2056 - accuracy: 0.9306
Epoch 2: saving model to ../model/mobilenetV2/mobilenetv2_cp\cp.ckpt
128/128 [==============================] - 363s 3s/step - loss: 0.2056 - accuracy: 0.9306 - val_loss: 0.1974 - val_accuracy: 0.9242
Epoch 3/25
128/128 [==============================] - ETA: 0s - loss: 0.1584 - accuracy: 0.9490
Epoch 3: saving model to ../model/mobilenetV2/mobilenetv2_cp\cp.ckpt
128/128 [==============================] - 366s 3s/step - loss: 0.1584 - accuracy: 0.9490 - val_loss: 0.1905 - val_accuracy: 0.9286
Epoch 4/25
128/128 [==============================] - ETA: 0s - loss: 0.1326 - accuracy: 0.9584
Epoch 4: savi

In [17]:
hist_df = pd.DataFrame(history.history) 
# save history to csv:  
hist_csv_file = '../model/mobilenetV2/history_mobilenetv2_tf.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [18]:
model.save("../model/mobilenetV2/mobilenetv2.h5")

In [ ]:
plot_graph(history)